# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [41]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [42]:
# read in csv output file from WeatherPy
output_data_file = "../output_data/cities.csv"

df = pd.read_csv(output_data_file)
df.head()


,City,Lat,Lon,Temp,Humidity,Cloudiness,Windspeed
0,Carnarvon,-24.87,113.63,73.40,69,40,16.11
1,Ushuaia,-54.80,-68.30,57.20,47,40,20.80
2,Narsaq,60.92,-46.05,28.40,74,100,3.36
3,Borogontsy,62.67,131.17,-10.08,89,97,4.99
4,Busselton,-33.65,115.33,66.00,83,97,1.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [68]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations and Humidity as weight
locations = df[["Lat", "Lon"]]
heatWeight = df["Humidity"]

In [70]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=heatWeight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:
# Filter wind speed
filtered_df = df.loc[df["Windspeed"] < 10]
# Updated count
filtered_df.count()

City          384
Lat           384
Lon           384
Temp          384
Humidity      384
Cloudiness    384
Windspeed     384
dtype: int64

In [46]:
# Filter down to cities with zero cloudiness
cloud_df = filtered_df.loc[filtered_df["Cloudiness"] == 0]
cloud_df.count()

City          88
Lat           88
Lon           88
Temp          88
Humidity      88
Cloudiness    88
Windspeed     88
dtype: int64

In [57]:
# Filter to temps above 70
temp_df = cloud_df.loc[cloud_df["Temp"] > 70]
temp_df.count()

City          20
Lat           20
Lon           20
Temp          20
Humidity      20
Cloudiness    20
Windspeed     20
dtype: int64

In [58]:
final_df = temp_df.loc[temp_df["Temp"] <= 80]
final_df.count()

City          13
Lat           13
Lon           13
Temp          13
Humidity      13
Cloudiness    13
Windspeed     13
dtype: int64

In [61]:
# Create lists to hold hotel data
latitudes = final_df["Lat"].tolist()
longitudes = final_df["Lon"].tolist()
cities = final_df["City"].tolist()
hotels = []
hotel_locations = []
hotel_cities = []
hotel_lat = []
hotel_lng = []

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [62]:
for x in range (len(latitudes)):
    target_coordinates = f"{latitudes[x]}, {longitudes[x]}"
    
    params = {
        "location": target_coordinates,
        "keyword": "Hotel",
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
     # try to pull in hotel data for each location
    try:
        hotels.append(response_json["results"][0]["name"])
        hotel_locations.append(response_json["results"][0]["plus_code"]["compound_code"])
        hotel_lat.append(latitudes[x])
        hotel_lng.append(longitudes[x])
        hotel_cities.append(cities[x])
        print(f"Hotel Found. Name Added.")
    # handle exceptions if hotel data is not available for location    
    except:
        print(f"No hotels found. Skipping")
        pass
    

Hotel Found. Name Added.
No hotels found. Skipping
Hotel Found. Name Added.
Hotel Found. Name Added.
Hotel Found. Name Added.
Hotel Found. Name Added.
No hotels found. Skipping
Hotel Found. Name Added.
No hotels found. Skipping
Hotel Found. Name Added.
Hotel Found. Name Added.
Hotel Found. Name Added.
Hotel Found. Name Added.


In [72]:
data_dict = {
    "Lat" : hotel_lat,
    "Lng" : hotel_lng,
    "City" : hotel_cities,
    "Country" : hotel_locations,
    "Hotel Name" : hotels,
}

hotel_df = pd.DataFrame(data_dict)
hotel_df

,Lat,Lng,City,Country,Hotel Name
0,-39.03,177.37,Wairoa,"WCQ4+FJ Wairoa, New Zealand",Whakamahia Lodge
1,17.02,54.09,Salalah,"2496+Q7 Salalah, Oman",HYATT SALALAH HOTEL
2,16.78,94.73,Pathein,"RQ29+Q3 Lon Kan Ward, Myanmar (Burma)",HOTEL LONKAN
3,20.43,99.88,Mae Sai,"CV4R+P4 Wiang Phang Kham, Mae Sai District, Ch...",KhuanJai Resort
4,4.93,-52.33,Cayenne,"WMMW+W4 Cayenne, French Guiana",Royal Amazonia
5,10.30,1.38,Natitingou,"899F+7G Natitingou, Benin",Hotel Totora
6,-28.78,32.04,Richards Bay,"634J+M8 Richards Bay, South Africa",BON Hotel Waterfront Richards Bay
7,-20.93,29.00,Gwanda,"3XJV+M7 Gwanda, Zimbabwe",Skmo Lodge
8,-17.88,36.89,Quelimane,"4VGP+P8 Quelimane, Mozambique",Hotel Elite
9,25.12,62.33,Gwadar,"483H+HF Gwadar, Pakistan",Zaver Pearl Continental Hotel


In [73]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [74]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))